In [24]:
import math

class Location:
    def __init__(self, lat, lon):
        self.__lat = lat
        self.__lon = lon
    def get_lat(self):
        return self.__lat
    def get_lon(self):
        return self.__lon
    def set_precision(self, precision):
        lt = round(self.__lat, precision)
        ln = round(self.__lon, precision)
        loc = Location(lt, ln)
        return loc
    def __degToRad(self, deg):
        return deg * math.pi / 180
    def distance(self, point):
        R = 6371e3
        lat1 = self.__degToRad(self.__lat)
        lat2 = self.__degToRad(point.get_lat())
        deltaLat = self.__degToRad(point.get_lat() - self.__lat)
        deltaLon = self.__degToRad(point.get_lon() - self.__lon)
        a = math.sin(deltaLat / 2) * math.sin(deltaLat / 2) + math.cos(lat1) * math.cos(lat2) * math.sin(deltaLon / 2) * math.sin(deltaLon / 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

In [25]:
class Region:
    def __init__(self, lat_range, lon_range):
        self.__lat_range = lat_range
        self.__lon_range = lon_range
        self.__hotspots = []
        self.__frequency = dict()
    def get_range(self):
        return self.__lat_range, self.__lon_range
    def get_neighbours(self):
        pass
    def get_hotspots(self):
        return self.__hotspots
    def __sort_hotspots(self):
        self.__hotspots.sort(key=lambda x: self.__frequency[x], reverse=True)
    def add_spot(self, spot):
        spot = spot.set_precision(4)
        spot = (spot.get_lat(), spot.get_lon())
        idx = self.__hotspots.index(spot) if spot in self.__hotspots else -1
        # print(idx)
        if idx == -1:
            self.__hotspots.append(spot)
            self.__frequency[spot] = 1
        else:
            self.__frequency[spot] += 1
        self.__sort_hotspots()


In [50]:
degree = 180
lat1000 = 0.009
lon1000 = 0.0147
# lat1000 = 1
# lon1000 = 1
lat_cells = int(degree // lat1000) * 2
lon_cells = int(degree // lon1000) * 2
lat_mid = lat_cells // 2
lon_mid = lon_cells // 2

print("Lat Cells:", lat_cells) 
print("Lon Cells:", lon_cells)
print("Total Regions:", lat_cells * lon_cells)
print("Max Locations in a Region:", math.ceil((lat1000 / .0001) * (lon1000 / .0001)))

Lat Cells: 40000
Lon Cells: 24488
Total Regions: 979520000
Max Locations in a Region: 13230


In [49]:
world_map = []
lat = -180
for i in range(lat_cells):
    lon = -180
    world_map.append([])
    for j in range(lon_cells):
        world_map[i].append(Region((lat, lat + lat1000), (lon, lon + lon1000)))
        lon += lon1000
    lat += lat1000

In [28]:
def location_to_index(loc):
    lat = loc.get_lat()
    lon = loc.get_lon()
    lat_idx = math.floor(lat // lat1000) + lat_mid
    lon_idx = math.floor(lon // lon1000) + lon_mid
    return (lat_idx, lon_idx)

In [29]:
def get_obfuscated_loc(loc, range):
    idx = location_to_index(loc)
    r = world_map[idx[0]][idx[1]]
    hotspots = r.get_hotspots()
    for spot in hotspots:
        if range[0] <= loc.distance(Location(spot[0], spot[1])) <= range[1]:
            return Location(spot[0], spot[1])

In [34]:
pos = list()
pos.append(Location(32.758, 74.89639)) # dist = 0
pos.append(Location(32.758469, 74.896626)) # dist = 50+
pos.append(Location(32.757593, 74.895512)) # dist = 90+
pos.append(Location(32.759156, 74.895684)) # dist = 140+
pos.append(Location(32.762922, 74.895594)) # dist = 500+
pos.append(Location(32.764816, 74.897932)) # dist = 700+
print(pos[0].distance(pos[5]))

771.4983233777218


In [35]:
idx = location_to_index(pos[3])
print("Index:", idx)
r = world_map[idx[0]][idx[1]]
print("Lat-Lon Range:", r.get_range())

Index: (212, 254)
Lat-Lon Range: ((32, 33), (74, 75))


In [36]:
for p in pos[1: ]:
    idx = location_to_index(p)
    r = world_map[idx[0]][idx[1]]
    r.add_spot(p)

-1
-1
-1
-1
-1


In [42]:
ob_l = get_obfuscated_loc(pos[0], (50, 100))
if ob_l: 
    print(ob_l.get_lat(), ob_l.get_lon())
    print(pos[0].distance(ob_l))
    print()
ob_l = get_obfuscated_loc(pos[0], (100, 150))
if ob_l: 
    print(ob_l.get_lat(), ob_l.get_lon())
    print(pos[0].distance(ob_l))
    print()
ob_l = get_obfuscated_loc(pos[0], (150, 500))
if ob_l: 
    print(ob_l.get_lat(), ob_l.get_lon())
    print(pos[0].distance(ob_l))
    print()
ob_l = get_obfuscated_loc(pos[0], (500, 1000))
if ob_l: 
    print(ob_l.get_lat(), ob_l.get_lon())
    print(pos[0].distance(ob_l))
    print()

32.7585 74.8966
58.963536580550524

32.7592 74.8957
148.21506648370803

32.7629 74.8956
549.8400958017124

